#### Example queries from Multi-omic provider.

##### This jupyter notebook give an example to query data from disease specific gene  mutation

In [1]:
import requests

import pandas as pd
def Query_fields_Mut_Freq_KP():
    Query_str = "https://biothings.ncats.io/tcga_mut_freq_kp/metadata/fields"
    response = requests.get(Query_str)
    result = response.json()
    return(result)

def format_result_Mut_Freq(result):
    subject_id = []
    subject_symbol = []
    subject_type = []

    object_id = []
    object_name = []
    object_type = []

    edge_context = []
    edge_label = []
    edge_confidence_freq_by_case = []
    edge_confidence_freq_by_sample = []
    
    for i in range(0,len(result['hits'])):
        #Edges 
        edge_label.append(result['hits'][i]['association']['edge_label'])
        edge_confidence_freq_by_case.append(result['hits'][i]['association']['freq_by_case'])
        edge_confidence_freq_by_sample.append(result['hits'][i]['association']['freq_by_sample'])
     
        #Subject
        subject_id.append(result['hits'][i]['subject']['id'])  
        subject_symbol.append(result['hits'][i]['subject']['SYMBOL'])
        subject_type.append(result['hits'][i]['subject']['type'])

        #Object
        object_id.append(result['hits'][i]['object']['id'])
        object_name.append(result['hits'][i]['object']['tcga_name'])
        object_type.append(result['hits'][i]['object']['type'])

    #Generating dataFrame for the results
    result_df = pd.DataFrame({
    "subject_id":subject_id,
    "subject_symbol": subject_symbol,
    "subject_type": subject_type,
        
    "object_id":object_id,
    "object_name":object_name,
    "object_type":object_type,
        
    "edge_label": edge_label,
    "edge_freq_by_case": edge_confidence_freq_by_case,
    "edge_freq_by_sample":edge_confidence_freq_by_sample})
    return(result_df)


def Query_Mut_Freq_KP(Query):
    Query_str = "https://biothings.ncats.io/tcga_mut_freq_kp/query?q="
    
    count = 0
    for i in Query:
        count = count + 1
        if count != len(Query):
            Query_str = Query_str + i + ":"+ Query[i] + "%20AND%20"
        else:
            Query_str = Query_str + i + ":"+ Query[i]+"&size=1000"
    print(Query_str)
    response = requests.get(Query_str)
    result = response.json()
    return(result)

#### Browse the field info in the KG

In [2]:
#Get to know the fields in the KG
meta = Query_fields_Mut_Freq_KP()
(meta)

{'association': {'index': True, 'type': 'object'},
 'association.edge_label': {'index': True, 'type': 'keyword'},
 'association.freq_by_case': {'index': True, 'type': 'float'},
 'association.freq_by_sample': {'index': True, 'type': 'float'},
 'association.no_mut_cases': {'index': True, 'type': 'integer'},
 'association.no_mut_samples': {'index': True, 'type': 'integer'},
 'association.relation_name': {'index': True, 'type': 'keyword'},
 'object': {'index': True, 'type': 'object'},
 'object.MONDO': {'index': True,
  'searched_by_default': True,
  'type': 'keyword'},
 'object.id': {'index': True, 'type': 'keyword'},
 'object.tcga_name': {'index': True, 'type': 'keyword'},
 'object.type': {'index': True, 'type': 'keyword'},
 'subject': {'index': True, 'type': 'object'},
 'subject.SYMBOL': {'index': True,
  'searched_by_default': True,
  'type': 'keyword'},
 'subject.id': {'index': True, 'type': 'keyword'},
 'subject.type': {'index': True, 'type': 'keyword'}}

#### <center>Query data from the drug_response_kp -- Example 1</center>
<font color=red><center>---------------------User input-------------------------</center></font>

In [3]:
#https://www.w3schools.com/tags/ref_urlencode.ASP

Query1 = {"object.id":"%22MONDO:0018177%22",
         "association.freq_by_case":"%3E0.03",
         #"association.freq_by_case":"%3E0.1",
         #"subject.SYMBOL":"EGFR"
        }

In [ ]:
Query1 = {"object.id":"%22MONDO:0018177%22",
         "association.freq_by_case":"%3E0.03"
        }

In [4]:
result = Query_Mut_Freq_KP(Query1)  #Jason format
result_df = format_result_Mut_Freq(result) #Pandas DataFrame format
result_df.sort_values(by = ['edge_freq_by_case'], ascending = False)

https://biothings.ncats.io/tcga_mut_freq_kp/query?q=object.id:%22MONDO:0018177%22%20AND%20association.freq_by_case:%3E0.03&size=1000


,subject_id,subject_symbol,subject_type,object_id,object_name,object_type,edge_label,edge_freq_by_case,edge_freq_by_sample
33,SYMBOL:PTEN,PTEN,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.328244,0.330025
38,SYMBOL:TP53,TP53,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.310433,0.315136
39,SYMBOL:TTN,TTN,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.254453,0.250620
78,SYMBOL:EGFR,EGFR,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.234097,0.235732
127,SYMBOL:MUC16,MUC16,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.150127,0.153846
...,...,...,...,...,...,...,...,...,...
107,SYMBOL:SPEG,SPEG,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.030534,0.029777
53,SYMBOL:TMEM132D,TMEM132D,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.030534,0.029777
41,SYMBOL:SLCO6A1,SLCO6A1,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.030534,0.032258
24,SYMBOL:CD163L1,CD163L1,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.030534,0.029777


##### From the gene muation frequency of EGFR, we observed EGFR is highly frequently mutated in GBM, LUAD, SKCM, LGG, UCEC, STAD. Afatinib is  approved for the treatment of NSCLC. The U.S. Food and Drug Administration (FDA) has approved a supplemental New Drug Application (sNDA) for Gilotrif® (afatinib) for the first-line treatment of patients with metastatic non-small cell lung cancer (NSCLC) whose tumors have non-resistant epidermal growth factor receptor (EGFR) mutations as detected by an FDA-approved test. The new label includes data on three additional EGFR mutations: L861Q, G719X and S768I. Our result may also suggest the potential application of Afatinib in other disease or tumor types. 

#### <center>Query data from the drug_response_kp -- Example 2</center>
<font color=red><center>---------------------User input-------------------------</center></font>

In [5]:
Query2 = {"object.id":"%22MONDO:0018177%22",
         "association.freq_by_case":"%3E0.05",
         #"association.freq_by_case":"%3E0.1",
         #"subject.SYMBOL":"EGFR"
        }

In [6]:
result = Query_Mut_Freq_KP(Query2)  #Jason format
result_df = format_result_Mut_Freq(result) #Pandas DataFrame format
result_df.sort_values(by = ['edge_freq_by_case'], ascending = False)

https://biothings.ncats.io/tcga_mut_freq_kp/query?q=object.id:%22MONDO:0018177%22%20AND%20association.freq_by_case:%3E0.05&size=1000


,subject_id,subject_symbol,subject_type,object_id,object_name,object_type,edge_label,edge_freq_by_case,edge_freq_by_sample
11,SYMBOL:PTEN,PTEN,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.328244,0.330025
13,SYMBOL:TP53,TP53,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.310433,0.315136
14,SYMBOL:TTN,TTN,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.254453,0.250620
24,SYMBOL:EGFR,EGFR,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.234097,0.235732
33,SYMBOL:MUC16,MUC16,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.150127,0.153846
27,SYMBOL:FLG,FLG,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.134860,0.136476
5,SYMBOL:NF1,NF1,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.122137,0.119107
34,SYMBOL:RYR2,RYR2,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.106870,0.106700
10,SYMBOL:PIK3R1,PIK3R1,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.099237,0.096774
26,SYMBOL:ATRX,ATRX,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.096692,0.096774
